In [ ]:
import pandas as pd
import os

Timebudgets are exported as indidivual files per observation id. Merge them all into one and add an ID keeping the origin.

These contain all behaviors for all subjects, even if the subject didn't exist in the specific observation or it didn't show the behavior. So we remove all entries where the "Total number of occurences" is 0.

There are some observations where two cameras where used, they have _a and _b suffixes that should be removed from the ID.

In [ ]:
TIME_BUDGET_FOLDER = "data export/data_export_30_sec_cut_off/time budget/single xlsx"

def create_unique_id(df: pd.DataFrame):
    df["uid"] = df["Observation id"].str.rstrip("_ab").str.cat(df["Subject"], sep="_")
    return df

def remove_nonexistant(df: pd.DataFrame):
    return df[df["Total number of occurences"] > 0]

files = map(lambda x: os.path.join(TIME_BUDGET_FOLDER, x), os.listdir(TIME_BUDGET_FOLDER))
excels = map(pd.read_excel, files)
filtered = map(remove_nonexistant, excels)
with_id = map(create_unique_id, filtered)

merged_timebudget = pd.concat(with_id)
merged_timebudget

In [ ]:
aggregated_events = pd.read_excel(
    "data export/data_export_30_sec_cut_off/aggregated events/aggregated_events_27.08.25.xlsx"
)
agg = create_unique_id(aggregated_events)

counted = agg.pivot_table(index=["uid", "Observation id"], columns="Behavior", aggfunc="size", fill_value=0)
counted

In [ ]:
plan = pd.read_excel("data export/playback_plan_filled_out.xlsx")
plan["Observation id"] = plan["experiment number"].astype(str).str.zfill(2).str.cat(plan["date"].dt.strftime("%d.%m.%Y"), sep="_")
plan = plan.set_index("Observation id")
plan = plan[["stim. cat.", "stimulus"]]
plan

In [ ]:
counted.join(plan, "Observation id", "left")